In [1]:
import socket
import sys
from collections import namedtuple
from struct import unpack
import asyncore
import rospy
from std_msgs.msg import Float32
from std_msgs.msg import String
from std_srvs.srv import Trigger, SetBool
import std_srvs.srv

In [2]:
Message = namedtuple('Message', ['position1','speed1','torque1','position2','speed2','torque2','first_letter','second_letter'])

rospy.init_node('start_stop_server')
class HTTPClient(asyncore.dispatcher):

    def __init__(self, host="192.168.1.120", port=8000, message_size=50):
  
        asyncore.dispatcher.__init__(self)
        self.create_socket()
        self.message_size=message_size #message_size=50
        self.connect( (host, port) ) #connect to port 8000, host 192.168.1.120 
        #self.buffer = bytes('testing','utf8')
        self.counter=0
        self.s_start = rospy.Service('start_stop', SetBool, self.start_stop_communication)      
        #self.s_mode = rospy.Service('mode', SetBool, self.change_mode)
        self.last_message=""
    
    def start_stop_communication(self,message):
        'this methond stops communication between ROS and the PLC'
        print("service response")
        print(message)
        #self.shutdown(self.SHUT_RDWR)
        self.close()
        return std_srvs.srv.SetBoolResponse(1, "Communication shutdown")

    #def change_mode(self):
        #return ModeResponse(True, "Mode changed successfully")
        
    def handle_connect(self):
        print("connected")
        pass
        
    def decode_message(self,message):
        'this method decodes the message received through TCP/IP'
        if(len(message)==self.message_size):
            touple_from_tcp=unpack('ddddddcc', message)
            my_message=Message(*touple_from_tcp)
            return my_message
        return False
    
    def handle_read(self):
        'this method receives a message through TCP/IP and sends it to ROS'
        byte_message=self.recv(self.message_size)
        self.last_message=self.decode_message(byte_message)
        if(self.last_message):
            try:
                pass
                #print(self.last_message.position1)
                #self.pub0.publish(self.last_message.position1)
                #self.pub1.publish(self.last_message.speed1)
                #self.pub2.publish(self.last_message.torque1)
                #self.pub3.publish(self.last_message.position2)
                #self.pub4.publish(self.last_message.speed2)
                #self.pub5.publish(self.last_message.torque2)
            except:
                print(":-(")
        self.counter+=1
        
    #def writable(self):
        #return (len(self.buffer) > 0)

    #def handle_write(self):
        #sent = self.send(self.buffer)
        #self.buffer = self.buffer[sent:]


In [3]:
client = HTTPClient('192.168.1.120',8000)
asyncore.loop()

connected
service response
data: False
service response
data: True


In [4]:
asyncore.loop()

In [2]:
#received_message=sock.recv(50)
#print(received_message)

In [3]:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        ip="192.168.1.120"
        port=8000
    # Connect the socket to the port where the server is listening
        server_address = (ip,port)
        print( 'connecting to %s port %s' % server_address)
        somesocket=sock.connect(server_address)
        #sock.sendall("testing".encode())
        received_message=sock.recv(50)



connecting to 192.168.1.120 port 8000


In [4]:
print(len(received_message))

50


In [5]:
touple_from_tcp=unpack('ddddddcc', received_message)
touple_from_tcp

(140.29997355707184, 0.0, -31.1, 150.69999303499608, 0.0, -4.5, b'O', b'K')

In [6]:
Message = namedtuple('Message', ['position1','speed1','torque1','position2','speed2','torque2','first_letter','second_letter'])
my_message=Message(*touple_from_tcp)
list(my_message)

[140.29997355707184, 0.0, -31.1, 150.69999303499608, 0.0, -4.5, b'O', b'K']

In [7]:
my_message.position1

140.29997355707184

In [ ]:
client.socket.close()

In [ ]:
client.counter

In [13]:
#!/usr/bin/env python

import rospy

def start_stop_communication(inside):
    #start stop rail 
    print("inside service response")
    print(inside)
    return std_srvs.srv.SetBoolResponse(1,"we have suceeded")
    



In [7]:
? std_srvs.srv.SetBoolResponse

In [14]:
s_start.shutdown()

In [15]:
s_start = rospy.Service('start_stop4', SetBool, start_stop_communication)


In [16]:
trigger_experiment = rospy.ServiceProxy('start_stop4', SetBool)

In [21]:
response=trigger_experiment(True)

inside service response
data: True


In [22]:
print(response.message)
print(response.success)


we have suceeded
True


In [20]:
sth=Trigger()

In [19]:
print(sth.)

In [21]:
? sth